### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import time 

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

### Getting target list 

In [2]:
datatracks = pd.read_csv('../data/clean/api_tracks1_clean.csv', encoding='latin1')

In [3]:
datatracks.head(2)

,track_id,track_name,album_id,artist_id,artist_name
0,4ZY1RPoLfezEqu03BI2axd,Sgt. Pepper's Lonely Hearts Club Band,1x1jpjDbetGqX0IKCUIBNj,3wGWCP3E3tYqj5memYV9Vq,Bloco do Sargento Pimenta
1,0QLyr8xasb6tgGkdPWYw8d,With a Little Help From My Friends,1x1jpjDbetGqX0IKCUIBNj,3wGWCP3E3tYqj5memYV9Vq,Bloco do Sargento Pimenta


In [6]:
track_ids = datatracks['track_id'].unique().tolist()

In [7]:
track_ids[0:2]

['4ZY1RPoLfezEqu03BI2axd', '0QLyr8xasb6tgGkdPWYw8d']

### Spotify Connection 

In [8]:
#%pip install pyarrow
#%pip install joblib
#%pip install tqdm
#%pip install spotipy
#%pip install python-dotenv
#%pip install requests 

In [9]:
import requests as rq
import pyarrow
from joblib import Parallel, delayed
from tqdm import tqdm
import pymongo
from passwords import *

In [10]:
# Obtener el TOKEN

AUTH_URL = "https://accounts.spotify.com/api/token"

creds = {
    "grant_type": "client_credentials",
    "client_id": Client_ID,
    "client_secret": Client_secret
}

response = rq.post(AUTH_URL, 
                         data=creds, 
                         headers={"Content-Type": "application/x-www-form-urlencoded"})


TOKEN = response.json()['access_token']

# Conexión con la API

url = "https://api.spotify.com/"

headers = {"Authorization": f'Bearer {TOKEN}'}

response = rq.get(url,headers=headers)
response

<Response [200]>

### Getting Tracks Information 

In [48]:
track_ids = datatracks['track_id'].unique().tolist()

In [11]:
track_ids[0:2]

['4ZY1RPoLfezEqu03BI2axd', '0QLyr8xasb6tgGkdPWYw8d']

In [17]:
#endpoint
#https://api.spotify.com/v1/audio-features/{id}

In [22]:
#test 
url = 'https://api.spotify.com/v1/audio-features/0QLyr8xasb6tgGkdPWYw8d'

In [30]:
res = rq.get(url, headers = headers)

In [31]:
resjson = res.json()
type(resjson)

dict

In [32]:
resjson.keys()

dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

In [29]:
resjson['danceability']

0.42

In [ ]:
# Construct a list of URLs for querying track audio features
track_feature_urls = [f'https://api.spotify.com/v1/audio-features/{track_id}' for track_id in track_ids]

# Create a list of dictionaries with track information and corresponding feature URLs
track_feature_url_list = [{'track_id': track_id, 'url': url} for track_id, url in zip(track_ids, track_feature_urls)]

In [ ]:
# List to store the retrieved track data
track_data_list = []

# Function to search and retrieve track information
def search_track_info(track_url):
    time.sleep(0.30)  # Pause between queries to avoid getting banned

    try:
        response = rq.get(track_url['url'], headers=headers)
        response.raise_for_status()  # Check for request errors
        data = response.json()

        # Extract necessary data from the response
        track_data_list.append({
            'track_id': data['id'],
            'track_name': data['name'],
            'album_id': data['album']['id'],
            'album_name': data['album']['name'],
            'artist_id': data['artists'][0]['id'],
            'artist_name': data['artists'][0]['name'],
            'danceability': data.get('danceability'),
            'popularity': data.get('popularity'),
            'energy': data.get('energy'),
            'acousticness': data.get('acousticness'),
            'valence': data.get('valence'),
            'instrumentalness': data.get('instrumentalness'),
            'tempo': data.get('tempo'),
            'loudness': data.get('loudness')
        })

    except Exception as e:
        print(f"An error occurred: {e}")
        track_data_list.append({
            'track_id': None,
            'track_name': None,
            'album_id': None,
            'album_name': None,
            'artist_id': None,
            'artist_name': None,
            'danceability': None,
            'popularity': None,
            'energy': None,
            'acousticness': None,
            'valence': None,
            'instrumentalness': None,
            'tempo': None,
            'loudness': None
        })

# Execute the function for each track URL
for track_url in track_url_list:
    search_track_info(track_url)

# Create a DataFrame from the list of dictionaries
data_tracks = pd.DataFrame(track_data_list)

In [74]:
data_tracks2.head()

,track_id,track_name,album_id,artist_id,artist_name
0,4ZY1RPoLfezEqu03BI2axd,Sgt. Pepper's Lonely Hearts Club Band,1x1jpjDbetGqX0IKCUIBNj,3wGWCP3E3tYqj5memYV9Vq,Bloco do Sargento Pimenta
1,0QLyr8xasb6tgGkdPWYw8d,With a Little Help From My Friends,1x1jpjDbetGqX0IKCUIBNj,3wGWCP3E3tYqj5memYV9Vq,Bloco do Sargento Pimenta
2,1YFlTlQHKR62Sh39ctI7jX,Lucy in the Sky with Diamonds,1x1jpjDbetGqX0IKCUIBNj,3wGWCP3E3tYqj5memYV9Vq,Bloco do Sargento Pimenta
3,1aBpFQAum7vESFuNnWeZZI,Getting Better,1x1jpjDbetGqX0IKCUIBNj,3wGWCP3E3tYqj5memYV9Vq,Bloco do Sargento Pimenta
4,2OePklaMhBp1kCbbTq5UVk,Fixing a Hole,1x1jpjDbetGqX0IKCUIBNj,3wGWCP3E3tYqj5memYV9Vq,Bloco do Sargento Pimenta


In [76]:
data_tracks.shape

(4818, 5)

In [77]:
data_tracks.to_csv('../data/clean/api_tracks1_clean.csv', index=False)